# Reproduce DL
## Automated Pavement Crack Segmentation

We start by setting up the actual Architecture. This means making sure all weights are properly initialized and all layers are connected. 

We make use of PyTorch for the implementation.

Multiple parts come together (A U-based ResNet);
- We recreate ResNet34 and remove the last two layers
- We made sure that a ResNet-block is either 4 or 6 layers depending on if stride is not 1 (which in our case always happens when the in_channels are not equal to out_channels)
- We use transfer learning such that the ResNet34 parameters are initialized as if trained on ImageNet
- We create Squeeze and Excitation blocks that are applied per Channel (cSE) and per Spatial (sSE) (image)
- These two blocks are combined (scSE) and then the maximum of this is taken
- Each convolutional layer its parameters are initialized via "He Kaiming" method.

In [1]:
# Do all the imports
## Packages
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torchsummary import summary
import torch
import pandas as pd


## Project
from architecture import main
from loss.loss import batch_dice_loss
from utils.layers import layer_split
from utils.retrieve_device import try_gpu
from utils.dataset import get_data_loaders
from evaluation.binary_classification import get_prec_recall, get_f1

In [ ]:
# Always needs to be a factor of 3
# Phase 1 = 1/3 time, Phase 2 = 2/3 time
EPOCHS = 90
epochs_1 = ( EPOCHS // 3 )
epochs_2 = ( EPOCHS // 3 ) * 2

# Define list to store losses and performances of each interation
metrics = []

# Try using gpu
device = try_gpu()

#Initialize network
network = main.Net()

#Initiliaze loss function
criterion = batch_dice_loss

# Split layers into three, for seperate optimization
layer_1, layer_2, layer_3 = layer_split(network)
datasetname = "CFD"

optimizer = torch.optim.AdamW([
    {'params': layer_1, 'name': 'layer_1'},
    {'params': layer_2, 'name': 'layer_2'},
    {'params': layer_3, 'name': 'layer_3'}], betas=(0.9, 0.999), weight_decay = 0.01)

# Image factor explanation:
# 1 = 320
# 0.8 = 256
# 0.4 = 128
split_seed = 42

# Get dataloaders (128x128)
dataset, train_loader, test_loader = get_data_loaders(split_seed, 0.4, datasetname)

# Look at this more carefully
# it should do this:
# - max lr is 1.0
# - start at 5% (0.05) of max_lr 
# - linearly build up
# - max_lr at (total_epoch * 0.4)
# - linearly break down
# - end at 0.00005 lr at last epoch
# Three phase: 
# - Up from initial to max,
# - Down from max to initial,
# - Down from initial to minimum
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=1.0,
    epochs=EPOCHS,
    steps_per_epoch=len(train_loader),
    anneal_strategy='linear',
    pct_start=0.4, 
    div_factor=20,
    final_div_factor=20000,
    three_phase=True
    )

for epoch in tqdm(range(EPOCHS)):
    # Network in training mode and to device
    network.train()
    network.to(device)
    
    optimizer.param_groups[0]["lr"] = 0 if epoch < epochs_1 else optimizer.param_groups[2]["lr"] / 9
    optimizer.param_groups[1]["lr"] = optimizer.param_groups[2]["lr"] / 3

    print(f"epoch: {epoch+1}, lr_layer_3 = {optimizer.param_groups[2]['lr']}")
    
    if (epoch == epochs_1):
        # Get dataloaders (256x256)
        dataset, train_loader, test_loader = get_data_loaders(0.8, datasetname)

    if (epoch == epochs_2):
        # Get dataloaders (320x320)
        dataset, train_loader, test_loader = get_data_loaders(1, datasetname)

    # Training loop
    for i, (x_batch, y_batch) in enumerate(train_loader):

        # Set to same device
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        # Set the gradients to zero
        optimizer.zero_grad()

        # Perform forward pass
        y_pred = network(x_batch)

        # Compute the loss
        loss = criterion(y_pred, y_batch)

        # Backward computation and update
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    # Compute precision, recall, and f1 for train and test data
    if epoch >= EPOCHS - 3:
        train_prec, train_recall = get_prec_recall(train_loader, network.to("cpu"))
        train_f1 = get_f1(train_prec, train_recall)
        test_prec, test_recall = get_prec_recall(test_loader, network.to("cpu"))
        test_f1 = get_f1(test_prec, test_recall)
        metrics.append([train_prec, train_recall, train_f1, test_prec, test_recall, test_f1, loss.tolist()])

        # Print performance
        print('Epoch: {:.0f}'.format(epoch+1))
        print(f'Precision, Recall, and F1 of train set: {train_prec}, {train_recall}, {train_f1}')
        print(f'Precision, Recall, and F1 of test set: {test_prec}, {test_recall}, {test_f1}')
        print('')


# Save model
model_state = network.state_dict()
reproduction_info = { "params": model_state, "split_seed": split_seed }
torch.save(model_state, "model_parameters.pt")

# Write metrics to disk
df = pd.DataFrame(metrics, columns=[
    "train_prec", "train_recall", "train_f1", "test_prec", "test_recall", "test_f1", "loss"])
df.to_csv("metrics.csv", index=False)

In [ ]:
# Summarize the Architecture as output
network = main.Net()
device = try_gpu()
model = network.to(device)

# print(network)
summary(model, input_size=(3, 320, 480))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 160, 240]           9,408
       BatchNorm2d-2         [-1, 64, 160, 240]             128
              ReLU-3         [-1, 64, 160, 240]               0
            Conv2d-4        [-1, 128, 160, 240]           8,320
         MaxPool2d-5          [-1, 64, 80, 120]               0
            Conv2d-6          [-1, 64, 80, 120]          36,864
       BatchNorm2d-7          [-1, 64, 80, 120]             128
              ReLU-8          [-1, 64, 80, 120]               0
            Conv2d-9          [-1, 64, 80, 120]          36,864
      BatchNorm2d-10          [-1, 64, 80, 120]             128
             ReLU-11          [-1, 64, 80, 120]               0
         ResBlock-12          [-1, 64, 80, 120]               0
           Conv2d-13          [-1, 64, 80, 120]          36,864
      BatchNorm2d-14          [-1, 64, 

In [ ]:
def epoch_to_PIL(data_loader, network, device=torch.device("cpu")) -> dict:
    converted = {"xs": [], "ys": [], "preds": []}
    for x_batch, y_batch in data_loader:
        preds = network(x_batch)
        converted["xs"].extend(batch_to_PIL(x_batch))
        converted["ys"].extend(batch_to_PIL(y_batch))
        converted["preds"].extend(batch_to_PIL(preds))
    return converted


def batch_to_PIL(tensor_batch) -> list:
    converted = []
    for t in tensor_batch:
        img = transforms.ToPILImage()(t)
        converted.append(img)
    return converted

network = main.Net()
model.load_state_dict(torch.load("model_parametersCFD.pt"))#["params"]
model.eval()

pics = epoch_to_PIL(test_loader, network)

subplot(r,c) # provide the no. of rows and columns
n_examples = 10
f, axarr = plt.subplots(n_examples,3, figsize=(15, n_examples*4)) 

for i in range(n_examples):
    axarr[i][0].imshow(pics["xs"][i])
    axarr[i][1].imshow(pics["ys"][i])
    axarr[i][2].imshow(pics["preds"][i])


dataset len: 118
dataset: 118, objects_train: 71, BS_train: 10, dataloader len: 8
objects_test: 47, BS_test: 47, dataloader len: 1


KernelInterrupted: Execution interrupted by the Jupyter kernel.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=411d58e9-cb4b-4924-bef0-2f383eff0187' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>